# OpenJij QUBO Solver Benchmark

This notebook provides a comprehensive benchmark of the OpenJij Simulated Quantum Annealing (SQA) solver for the Access Point (AP) selection problem.

## Benchmark Objectives

1. **Time to Solution (TTS)**: Measure how long it takes to find optimal solutions
2. **Solution Quality**: Evaluate energy levels and constraint satisfaction
3. **Parameter Sensitivity**: Test how different parameters affect performance
4. **Success Probability**: Calculate probability of finding optimal/near-optimal solutions

## Parameters to Benchmark

- **num_reads**: Number of times to run the solver (sampling count)
- **num_sweeps**: Number of Monte Carlo steps (annealing duration)
- **Building ID**: Test on different buildings
- **k**: Number of APs to select
- **alpha**: Importance vs redundancy trade-off

## 1. Setup and Imports

In [ ]:
# Standard libraries
import openjij as oj
import time
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"OpenJij version: {oj.__version__}")
print("✓ Imports successful")

In [ ]:
# Import custom modules
from scripts.data.data_loaders import load_preprocessed_data, load_all_precomputed_data
from scripts.optimization.QUBO import formulate_qubo
from scripts.ml.ML_post_processing import train_regressor
from scripts.evaluation.Analysis import calculate_comprehensive_metrics

print("✓ Custom modules imported successfully")

## 2. Configuration Parameters

Configure all experiment parameters in this cell.

In [ ]:
# ============================================================================
# EXPERIMENT CONFIGURATION
# ============================================================================

# Building selection
BUILDING_ID = 1  # Options: 0, 1, 2

# QUBO parameters
K = 20  # Number of APs to select
ALPHA = 0.9  # Importance weight (0.0 to 1.0)
PENALTY = 2.0  # Constraint penalty
IMPORTANCE_METHOD = 'average'  # Options: 'entropy', 'average', 'max', 'variance', 'mutual_info'

# Benchmark parameters
NUM_READS_RANGE = [10, 50, 100, 200, 500, 1000]  # Different sampling counts
NUM_SWEEPS_RANGE = [100, 500, 1000, 2000, 5000]  # Different annealing durations

# Benchmark settings
NUM_REPETITIONS = 10  # Number of times to repeat each experiment for statistical significance
RANDOM_SEED = 42

# Floor height for evaluation
FLOOR_HEIGHT = 3.0

# Results output directory
OUTPUT_DIR = Path('data') / 'results' / 'benchmarks'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("="*80)
print("BENCHMARK CONFIGURATION")
print("="*80)
print(f"Building ID: {BUILDING_ID}")
print(f"Number of APs to select (k): {K}")
print(f"Alpha (importance weight): {ALPHA}")
print(f"Penalty: {PENALTY}")
print(f"Importance method: {IMPORTANCE_METHOD}")
print(f"\nBenchmark ranges:")
print(f"  num_reads: {NUM_READS_RANGE}")
print(f"  num_sweeps: {NUM_SWEEPS_RANGE}")
print(f"  Repetitions per experiment: {NUM_REPETITIONS}")
print(f"\nTotal experiments: {len(NUM_READS_RANGE) * len(NUM_SWEEPS_RANGE) * NUM_REPETITIONS}")
print(f"Output directory: {OUTPUT_DIR}")
print("="*80)

## 3. Load Data

In [ ]:
# Load preprocessed data for the selected building
print(f"Loading preprocessed data for Building {BUILDING_ID}...")
rssi_train, coords_train, rssi_val, coords_val, ap_columns = load_preprocessed_data(
    building_id=BUILDING_ID,
    use_pickle=True
)

print(f"\n✓ Data loaded successfully")
print(f"  Training samples: {len(rssi_train):,}")
print(f"  Validation samples: {len(rssi_val):,}")
print(f"  Number of APs: {len(ap_columns):,}")

In [ ]:
# Load importance scores and redundancy matrix
print("Loading importance scores and redundancy matrix...")
importance_dicts_loaded, redundancy_matrix_loaded = load_all_precomputed_data()

# Select the importance method
importance_dict = importance_dicts_loaded[IMPORTANCE_METHOD]

print(f"\n✓ Using '{IMPORTANCE_METHOD}' importance scores")
print(f"  Non-zero importance scores: {sum(1 for v in importance_dict.values() if v > 0)}/{len(importance_dict)}")

In [ ]:
# Load system parameters for denormalization
system_params_path = Path('data') / 'system_input' / f'system_parameters_building_{BUILDING_ID}.csv'
system_params_df = pd.read_csv(system_params_path)
system_params_dict = dict(zip(system_params_df['Parameter'], system_params_df['Value']))

LON_MIN = system_params_dict['LON_MIN']
LON_MAX = system_params_dict['LON_MAX']
LAT_MIN = system_params_dict['LAT_MIN']
LAT_MAX = system_params_dict['LAT_MAX']

print(f"\n✓ System parameters loaded")
print(f"  Longitude range: [{LON_MIN:.2f}, {LON_MAX:.2f}]")
print(f"  Latitude range: [{LAT_MIN:.2f}, {LAT_MAX:.2f}]")

## 4. QUBO Formulation

In [ ]:
# Formulate QUBO problem once (same for all benchmark runs)
print("Formulating QUBO problem...")
Q, relevant_aps, offset = formulate_qubo(
    importance_dict,
    redundancy_matrix_loaded,
    K,
    ALPHA,
    PENALTY
)

print(f"\n✓ QUBO formulated")
print(f"  Number of relevant APs: {len(relevant_aps)}")
print(f"  QUBO size: {len(Q)} terms")
print(f"  Offset: {offset:.4f}")

## 5. Benchmark Functions

In [ ]:
def solve_qubo_openjij(Q, num_reads=1000, num_sweeps=1000, seed=None):
    """
    Solve QUBO with OpenJij SQA sampler and return detailed results.
    
    Returns:
        dict: Contains solution, energy, timing, and other metrics
    """
    sampler = oj.SQASampler()
    
    start_time = time.time()
    response = sampler.sample_qubo(
        Q,
        num_reads=num_reads,
        num_sweeps=num_sweeps,
        seed=seed
    )
    end_time = time.time()
    
    # Extract best solution
    best_solution = response.first.sample
    best_energy = response.first.energy
    
    # Count selected APs
    num_selected = sum(best_solution.values())
    
    # Extract all energies and calculate statistics
    all_energies = [record.energy for record in response.record]
    
    return {
        'solution': best_solution,
        'energy': best_energy,
        'num_selected': num_selected,
        'time': end_time - start_time,
        'all_energies': all_energies,
        'min_energy': min(all_energies),
        'mean_energy': np.mean(all_energies),
        'std_energy': np.std(all_energies),
        'response': response
    }

def evaluate_solution_quality(solution, relevant_aps, rssi_train, coords_train, 
                              rssi_val, coords_val, LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, FLOOR_HEIGHT):
    """
    Evaluate the quality of a solution by training a model and measuring error.
    
    Returns:
        dict: Contains localization metrics
    """
    # Get selected APs
    selected_indices = [i for i, val in solution.items() if val == 1]
    selected_aps = [relevant_aps[i] for i in selected_indices]
    
    if len(selected_aps) == 0:
        return None
    
    try:
        # Train model
        models, predictions = train_regressor(rssi_train, coords_train, rssi_val, coords_val, selected_aps)
        preds = predictions['rf_val']
        
        # Calculate metrics
        _, _, metrics = calculate_comprehensive_metrics(
            coords_val, preds,
            LON_MIN, LON_MAX,
            LAT_MIN, LAT_MAX,
            FLOOR_HEIGHT
        )
        
        return {
            'mean_3d_error': metrics['real_mean_m'],
            'median_3d_error': metrics['real_median_m'],
            'floor_accuracy': metrics['floor_accuracy'],
            'num_aps': len(selected_aps)
        }
    except Exception as e:
        print(f"Error evaluating solution: {e}")
        return None

print("✓ Benchmark functions defined")

## 6. Run Benchmarks

### 6.1 Benchmark: Varying num_reads (fixed num_sweeps)

In [ ]:
# Benchmark varying num_reads with fixed num_sweeps
FIXED_NUM_SWEEPS = 1000

print("="*80)
print(f"BENCHMARK 1: Varying num_reads (num_sweeps={FIXED_NUM_SWEEPS})")
print("="*80)

benchmark_reads_results = []

for num_reads in tqdm(NUM_READS_RANGE, desc="num_reads"):
    for rep in range(NUM_REPETITIONS):
        seed = RANDOM_SEED + rep
        
        # Solve QUBO
        result = solve_qubo_openjij(Q, num_reads=num_reads, num_sweeps=FIXED_NUM_SWEEPS, seed=seed)
        
        # Record results
        benchmark_reads_results.append({
            'num_reads': num_reads,
            'num_sweeps': FIXED_NUM_SWEEPS,
            'repetition': rep,
            'time': result['time'],
            'energy': result['energy'],
            'min_energy': result['min_energy'],
            'mean_energy': result['mean_energy'],
            'std_energy': result['std_energy'],
            'num_selected': result['num_selected'],
            'constraint_satisfied': (result['num_selected'] == K)
        })

# Convert to DataFrame
df_reads = pd.DataFrame(benchmark_reads_results)

print(f"\n✓ Benchmark 1 complete: {len(df_reads)} experiments")
print(f"\nSummary statistics:")
print(df_reads.groupby('num_reads')[['time', 'energy', 'num_selected']].agg(['mean', 'std']))

### 6.2 Benchmark: Varying num_sweeps (fixed num_reads)

In [ ]:
# Benchmark varying num_sweeps with fixed num_reads
FIXED_NUM_READS = 100

print("="*80)
print(f"BENCHMARK 2: Varying num_sweeps (num_reads={FIXED_NUM_READS})")
print("="*80)

benchmark_sweeps_results = []

for num_sweeps in tqdm(NUM_SWEEPS_RANGE, desc="num_sweeps"):
    for rep in range(NUM_REPETITIONS):
        seed = RANDOM_SEED + rep
        
        # Solve QUBO
        result = solve_qubo_openjij(Q, num_reads=FIXED_NUM_READS, num_sweeps=num_sweeps, seed=seed)
        
        # Record results
        benchmark_sweeps_results.append({
            'num_reads': FIXED_NUM_READS,
            'num_sweeps': num_sweeps,
            'repetition': rep,
            'time': result['time'],
            'energy': result['energy'],
            'min_energy': result['min_energy'],
            'mean_energy': result['mean_energy'],
            'std_energy': result['std_energy'],
            'num_selected': result['num_selected'],
            'constraint_satisfied': (result['num_selected'] == K)
        })

# Convert to DataFrame
df_sweeps = pd.DataFrame(benchmark_sweeps_results)

print(f"\n✓ Benchmark 2 complete: {len(df_sweeps)} experiments")
print(f"\nSummary statistics:")
print(df_sweeps.groupby('num_sweeps')[['time', 'energy', 'num_selected']].agg(['mean', 'std']))

### 6.3 Benchmark: Parameter Grid (num_reads × num_sweeps)

In [ ]:
# Full grid benchmark (smaller ranges for efficiency)
GRID_NUM_READS = [50, 100, 200, 500]
GRID_NUM_SWEEPS = [500, 1000, 2000]
GRID_REPETITIONS = 5

print("="*80)
print("BENCHMARK 3: Parameter Grid (num_reads × num_sweeps)")
print("="*80)
print(f"Grid: {len(GRID_NUM_READS)} × {len(GRID_NUM_SWEEPS)} = {len(GRID_NUM_READS) * len(GRID_NUM_SWEEPS)} combinations")
print(f"Total experiments: {len(GRID_NUM_READS) * len(GRID_NUM_SWEEPS) * GRID_REPETITIONS}")

benchmark_grid_results = []

total = len(GRID_NUM_READS) * len(GRID_NUM_SWEEPS) * GRID_REPETITIONS
with tqdm(total=total, desc="Grid Search") as pbar:
    for num_reads in GRID_NUM_READS:
        for num_sweeps in GRID_NUM_SWEEPS:
            for rep in range(GRID_REPETITIONS):
                seed = RANDOM_SEED + rep
                
                # Solve QUBO
                result = solve_qubo_openjij(Q, num_reads=num_reads, num_sweeps=num_sweeps, seed=seed)
                
                # Record results
                benchmark_grid_results.append({
                    'num_reads': num_reads,
                    'num_sweeps': num_sweeps,
                    'repetition': rep,
                    'time': result['time'],
                    'energy': result['energy'],
                    'min_energy': result['min_energy'],
                    'mean_energy': result['mean_energy'],
                    'std_energy': result['std_energy'],
                    'num_selected': result['num_selected'],
                    'constraint_satisfied': (result['num_selected'] == K)
                })
                
                pbar.update(1)

# Convert to DataFrame
df_grid = pd.DataFrame(benchmark_grid_results)

print(f"\n✓ Benchmark 3 complete: {len(df_grid)} experiments")

### 6.4 Solution Quality Evaluation (on best configurations)

In [ ]:
# Evaluate solution quality for best configurations from grid search
print("="*80)
print("BENCHMARK 4: Solution Quality Evaluation")
print("="*80)

# Find best configurations (lowest average energy)
grid_summary = df_grid.groupby(['num_reads', 'num_sweeps'])['energy'].mean().reset_index()
grid_summary = grid_summary.sort_values('energy').head(5)

print("\nEvaluating top 5 configurations by energy:")
print(grid_summary)

quality_results = []

for _, row in tqdm(grid_summary.iterrows(), total=len(grid_summary), desc="Quality eval"):
    num_reads = int(row['num_reads'])
    num_sweeps = int(row['num_sweeps'])
    
    # Run solver
    result = solve_qubo_openjij(Q, num_reads=num_reads, num_sweeps=num_sweeps, seed=RANDOM_SEED)
    
    # Evaluate solution quality
    quality = evaluate_solution_quality(
        result['solution'], relevant_aps,
        rssi_train, coords_train,
        rssi_val, coords_val,
        LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, FLOOR_HEIGHT
    )
    
    if quality is not None:
        quality_results.append({
            'num_reads': num_reads,
            'num_sweeps': num_sweeps,
            'energy': result['energy'],
            'time': result['time'],
            'mean_3d_error': quality['mean_3d_error'],
            'median_3d_error': quality['median_3d_error'],
            'floor_accuracy': quality['floor_accuracy']
        })

df_quality = pd.DataFrame(quality_results)

print(f"\n✓ Quality evaluation complete")
print("\nQuality Results:")
print(df_quality.to_string(index=False))

## 7. Results Analysis and Visualization

### 7.1 Time to Solution Analysis

In [ ]:
# Plot time vs num_reads
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Time vs num_reads
df_reads_summary = df_reads.groupby('num_reads')['time'].agg(['mean', 'std']).reset_index()
axes[0].errorbar(df_reads_summary['num_reads'], df_reads_summary['mean'], 
                 yerr=df_reads_summary['std'], marker='o', capsize=5, linewidth=2)
axes[0].set_xlabel('Number of Reads', fontsize=12)
axes[0].set_ylabel('Time (seconds)', fontsize=12)
axes[0].set_title(f'Time vs num_reads (num_sweeps={FIXED_NUM_SWEEPS})', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Time vs num_sweeps
df_sweeps_summary = df_sweeps.groupby('num_sweeps')['time'].agg(['mean', 'std']).reset_index()
axes[1].errorbar(df_sweeps_summary['num_sweeps'], df_sweeps_summary['mean'], 
                 yerr=df_sweeps_summary['std'], marker='o', capsize=5, linewidth=2, color='orange')
axes[1].set_xlabel('Number of Sweeps', fontsize=12)
axes[1].set_ylabel('Time (seconds)', fontsize=12)
axes[1].set_title(f'Time vs num_sweeps (num_reads={FIXED_NUM_READS})', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'time_to_solution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Time to solution plots saved")

### 7.2 Energy Analysis

In [ ]:
# Plot energy vs parameters
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Energy vs num_reads
df_reads_energy = df_reads.groupby('num_reads')['energy'].agg(['mean', 'std', 'min']).reset_index()
axes[0].errorbar(df_reads_energy['num_reads'], df_reads_energy['mean'], 
                 yerr=df_reads_energy['std'], marker='o', capsize=5, linewidth=2, label='Mean ± Std')
axes[0].plot(df_reads_energy['num_reads'], df_reads_energy['min'], 
             marker='s', linestyle='--', linewidth=2, label='Min Energy')
axes[0].set_xlabel('Number of Reads', fontsize=12)
axes[0].set_ylabel('Energy', fontsize=12)
axes[0].set_title(f'Energy vs num_reads (num_sweeps={FIXED_NUM_SWEEPS})', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Energy vs num_sweeps
df_sweeps_energy = df_sweeps.groupby('num_sweeps')['energy'].agg(['mean', 'std', 'min']).reset_index()
axes[1].errorbar(df_sweeps_energy['num_sweeps'], df_sweeps_energy['mean'], 
                 yerr=df_sweeps_energy['std'], marker='o', capsize=5, linewidth=2, 
                 color='orange', label='Mean ± Std')
axes[1].plot(df_sweeps_energy['num_sweeps'], df_sweeps_energy['min'], 
             marker='s', linestyle='--', linewidth=2, color='red', label='Min Energy')
axes[1].set_xlabel('Number of Sweeps', fontsize=12)
axes[1].set_ylabel('Energy', fontsize=12)
axes[1].set_title(f'Energy vs num_sweeps (num_reads={FIXED_NUM_READS})', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'energy_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Energy analysis plots saved")

### 7.3 Parameter Grid Heatmap

In [ ]:
# Create heatmaps for grid search results
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Prepare pivot tables
pivot_time = df_grid.pivot_table(values='time', index='num_sweeps', columns='num_reads', aggfunc='mean')
pivot_energy = df_grid.pivot_table(values='energy', index='num_sweeps', columns='num_reads', aggfunc='mean')
pivot_constraint = df_grid.pivot_table(values='constraint_satisfied', index='num_sweeps', columns='num_reads', aggfunc='mean')

# Time heatmap
sns.heatmap(pivot_time, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[0], cbar_kws={'label': 'Time (s)'})
axes[0].set_title('Average Time to Solution', fontsize=14)
axes[0].set_xlabel('num_reads', fontsize=12)
axes[0].set_ylabel('num_sweeps', fontsize=12)

# Energy heatmap
sns.heatmap(pivot_energy, annot=True, fmt='.2f', cmap='RdYlGn_r', ax=axes[1], cbar_kws={'label': 'Energy'})
axes[1].set_title('Average Energy', fontsize=14)
axes[1].set_xlabel('num_reads', fontsize=12)
axes[1].set_ylabel('num_sweeps', fontsize=12)

# Constraint satisfaction heatmap
sns.heatmap(pivot_constraint, annot=True, fmt='.2%', cmap='RdYlGn', ax=axes[2], 
            cbar_kws={'label': 'Constraint Satisfaction Rate'})
axes[2].set_title('Constraint Satisfaction Rate', fontsize=14)
axes[2].set_xlabel('num_reads', fontsize=12)
axes[2].set_ylabel('num_sweeps', fontsize=12)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'parameter_grid_heatmaps.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Parameter grid heatmaps saved")

### 7.4 Time-Energy Trade-off

In [ ]:
# Plot time-energy trade-off
plt.figure(figsize=(12, 8))

# Aggregate grid results
grid_agg = df_grid.groupby(['num_reads', 'num_sweeps']).agg({
    'time': 'mean',
    'energy': 'mean',
    'constraint_satisfied': 'mean'
}).reset_index()

# Create scatter plot with color for constraint satisfaction
scatter = plt.scatter(grid_agg['time'], grid_agg['energy'], 
                     c=grid_agg['constraint_satisfied'], 
                     s=200, cmap='RdYlGn', edgecolors='black', linewidth=1.5,
                     alpha=0.7)

# Add labels for each point
for _, row in grid_agg.iterrows():
    plt.annotate(f"({int(row['num_reads'])}, {int(row['num_sweeps'])})",
                (row['time'], row['energy']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.colorbar(scatter, label='Constraint Satisfaction Rate')
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title('Time-Energy Trade-off\n(num_reads, num_sweeps)', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'time_energy_tradeoff.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Time-energy trade-off plot saved")

### 7.5 Solution Quality vs Time

In [ ]:
# Plot solution quality metrics
if len(df_quality) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Mean 3D error vs time
    axes[0].scatter(df_quality['time'], df_quality['mean_3d_error'], s=200, alpha=0.6, edgecolors='black')
    for idx, row in df_quality.iterrows():
        axes[0].annotate(f"({int(row['num_reads'])}, {int(row['num_sweeps'])})",
                        (row['time'], row['mean_3d_error']),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
    axes[0].set_xlabel('Time (seconds)', fontsize=12)
    axes[0].set_ylabel('Mean 3D Error (meters)', fontsize=12)
    axes[0].set_title('Localization Error vs Time', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    # Floor accuracy vs time
    axes[1].scatter(df_quality['time'], df_quality['floor_accuracy'] * 100, 
                   s=200, alpha=0.6, edgecolors='black', color='green')
    for idx, row in df_quality.iterrows():
        axes[1].annotate(f"({int(row['num_reads'])}, {int(row['num_sweeps'])})",
                        (row['time'], row['floor_accuracy'] * 100),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
    axes[1].set_xlabel('Time (seconds)', fontsize=12)
    axes[1].set_ylabel('Floor Accuracy (%)', fontsize=12)
    axes[1].set_title('Floor Accuracy vs Time', fontsize=14)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'quality_vs_time.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Solution quality plots saved")
else:
    print("No quality data available to plot")

## 8. Summary Statistics and Recommendations

In [ ]:
print("="*80)
print("BENCHMARK SUMMARY")
print("="*80)

# Overall statistics
print("\n1. OVERALL STATISTICS")
print("-" * 80)
print(f"Total experiments run: {len(df_reads) + len(df_sweeps) + len(df_grid)}")
print(f"Total computation time: {df_reads['time'].sum() + df_sweeps['time'].sum() + df_grid['time'].sum():.2f} seconds")

# Best configurations by energy
print("\n2. BEST CONFIGURATIONS BY ENERGY")
print("-" * 80)
best_energy_config = grid_agg.loc[grid_agg['energy'].idxmin()]
print(f"Lowest energy: {best_energy_config['energy']:.4f}")
print(f"  num_reads: {int(best_energy_config['num_reads'])}")
print(f"  num_sweeps: {int(best_energy_config['num_sweeps'])}")
print(f"  Time: {best_energy_config['time']:.2f} seconds")
print(f"  Constraint satisfied: {best_energy_config['constraint_satisfied']:.1%}")

# Best configurations by time
print("\n3. FASTEST CONFIGURATIONS (with constraint satisfaction)")
print("-" * 80)
satisfied = grid_agg[grid_agg['constraint_satisfied'] >= 0.8]  # At least 80% satisfaction
if len(satisfied) > 0:
    fastest_config = satisfied.loc[satisfied['time'].idxmin()]
    print(f"Fastest time: {fastest_config['time']:.2f} seconds")
    print(f"  num_reads: {int(fastest_config['num_reads'])}")
    print(f"  num_sweeps: {int(fastest_config['num_sweeps'])}")
    print(f"  Energy: {fastest_config['energy']:.4f}")
    print(f"  Constraint satisfied: {fastest_config['constraint_satisfied']:.1%}")
else:
    print("No configuration achieved 80% constraint satisfaction")

# Best configurations by quality (if available)
if len(df_quality) > 0:
    print("\n4. BEST CONFIGURATIONS BY LOCALIZATION QUALITY")
    print("-" * 80)
    best_quality_config = df_quality.loc[df_quality['mean_3d_error'].idxmin()]
    print(f"Lowest mean 3D error: {best_quality_config['mean_3d_error']:.2f} meters")
    print(f"  num_reads: {int(best_quality_config['num_reads'])}")
    print(f"  num_sweeps: {int(best_quality_config['num_sweeps'])}")
    print(f"  Median 3D error: {best_quality_config['median_3d_error']:.2f} meters")
    print(f"  Floor accuracy: {best_quality_config['floor_accuracy']:.1%}")
    print(f"  Time: {best_quality_config['time']:.2f} seconds")

# Recommendations
print("\n5. RECOMMENDATIONS")
print("-" * 80)
print("Based on the benchmark results:")
print(f"\n  • For BEST QUALITY: Use num_reads={int(best_energy_config['num_reads'])}, num_sweeps={int(best_energy_config['num_sweeps'])}")
if len(satisfied) > 0:
    print(f"  • For FASTEST RESULTS: Use num_reads={int(fastest_config['num_reads'])}, num_sweeps={int(fastest_config['num_sweeps'])}")
print(f"\n  • Increasing num_sweeps improves solution quality but increases computation time")
print(f"  • Increasing num_reads provides more sampling but has diminishing returns")
print(f"  • Balance between time and quality depends on application requirements")

print("\n" + "="*80)

## 9. Save Results

In [ ]:
# Save all benchmark results to CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save individual benchmark results
df_reads.to_csv(OUTPUT_DIR / f'benchmark_reads_{timestamp}.csv', index=False)
df_sweeps.to_csv(OUTPUT_DIR / f'benchmark_sweeps_{timestamp}.csv', index=False)
df_grid.to_csv(OUTPUT_DIR / f'benchmark_grid_{timestamp}.csv', index=False)

if len(df_quality) > 0:
    df_quality.to_csv(OUTPUT_DIR / f'benchmark_quality_{timestamp}.csv', index=False)

# Save summary
summary_data = {
    'timestamp': [timestamp],
    'building_id': [BUILDING_ID],
    'k': [K],
    'alpha': [ALPHA],
    'penalty': [PENALTY],
    'importance_method': [IMPORTANCE_METHOD],
    'total_experiments': [len(df_reads) + len(df_sweeps) + len(df_grid)],
    'best_energy': [best_energy_config['energy']],
    'best_num_reads': [int(best_energy_config['num_reads'])],
    'best_num_sweeps': [int(best_energy_config['num_sweeps'])],
    'best_time': [best_energy_config['time']]
}

if len(df_quality) > 0:
    summary_data['best_mean_3d_error'] = [best_quality_config['mean_3d_error']]
    summary_data['best_floor_accuracy'] = [best_quality_config['floor_accuracy']]

df_summary = pd.DataFrame(summary_data)
df_summary.to_csv(OUTPUT_DIR / f'benchmark_summary_{timestamp}.csv', index=False)

print("✓ All benchmark results saved to:")
print(f"  {OUTPUT_DIR}/")
print(f"\nFiles created:")
print(f"  - benchmark_reads_{timestamp}.csv")
print(f"  - benchmark_sweeps_{timestamp}.csv")
print(f"  - benchmark_grid_{timestamp}.csv")
if len(df_quality) > 0:
    print(f"  - benchmark_quality_{timestamp}.csv")
print(f"  - benchmark_summary_{timestamp}.csv")
print(f"  - time_to_solution.png")
print(f"  - energy_analysis.png")
print(f"  - parameter_grid_heatmaps.png")
print(f"  - time_energy_tradeoff.png")
if len(df_quality) > 0:
    print(f"  - quality_vs_time.png")

## Conclusion

This benchmark provides comprehensive insights into OpenJij's SQA solver performance for the AP selection problem. The results can guide parameter selection based on your specific requirements:

- **Research/Offline**: Use higher num_sweeps (2000+) for best quality
- **Real-time/Online**: Use lower num_sweeps (500-1000) for faster results
- **Balanced**: Use moderate settings (num_reads=100, num_sweeps=1000)

The benchmark can be re-run with different buildings, k values, or importance methods by changing the configuration parameters.